In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

### Cargo los datos

In [2]:
events = utils.get_events()

In [59]:
#events.head()

In [60]:
df  = utils.get_features_iniciales(events)
#df.head()

### Info de sesiones

In [6]:
# Tomo como sesión media hora
events = events.sort_values(by='timestamp')
events['timestamp_anterior'] = events.groupby('person')['timestamp'].shift()
events['start_session'] = ((events['timestamp'] - events['timestamp_anterior']) >= timedelta(minutes=30)) | events['timestamp_anterior'].isnull()
events['start_session'] = events['start_session'].astype(int)
events['session_num'] = events.groupby('person')['start_session'].cumsum()
events['session_id'] =  events['person'] + '-' + events['session_num'].astype(str)

In [7]:
session_df = events.groupby(['session_id','person']).agg({'event':'count', 'timestamp':['min','max']})
session_df.columns = session_df.columns.droplevel()
session_df.columns = ['number_events','timestamp_min','timestamp_max']
session_df['total_time'] = ((session_df['timestamp_max']) - session_df['timestamp_min']).astype('timedelta64[s]')
session_df = session_df.reset_index().sort_values(by='session_id')
session_df.head()

,session_id,person,number_events,timestamp_min,timestamp_max,total_time
0,0008ed71-1,0008ed71,1,2018-05-17 12:27:47,2018-05-17 12:27:47,0.0
1,0008ed71-2,0008ed71,2,2018-05-17 13:44:59,2018-05-17 13:45:00,1.0
2,0008ed71-3,0008ed71,3,2018-05-17 16:21:54,2018-05-17 16:28:37,403.0
3,00091926-1,00091926,7,2018-05-03 22:08:29,2018-05-03 22:09:59,90.0
4,00091926-10,00091926,4,2018-05-11 02:23:38,2018-05-11 02:23:57,19.0


In [8]:
sessions_by_person = session_df.groupby(['person']).agg({'session_id':'count', 'total_time':['sum','mean']})
sessions_by_person.columns = sessions_by_person.columns.droplevel()
sessions_by_person.columns = ['sessions', 'total_time', 'mean_time_by_session']
sessions_by_person = sessions_by_person.reset_index()
con_sesiones = utils.add_features(df,sessions_by_person)

In [9]:
con_sesiones.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,sessions,total_time,mean_time_by_session
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2,3,404.0,134.666667
1,00091926,15,25,2,0,0,0,0,0,0,372,34,34,14890.0,437.941176
2,00091a7a,1,5,0,0,0,0,0,0,0,3,1,1,347.0,347.000000
3,000ba417,1,24,6,1,14,0,1,0,0,153,6,5,12703.0,2540.600000
4,000c79fe,1,0,1,0,1,0,1,9,0,3,1,1,620.0,620.000000


In [46]:
sesiones = events.groupby(['session_id'])['event'].value_counts().unstack(fill_value=0)
sesiones.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
session_id,,,,,,,,,,,
0008ed71-1,0,0,1,0,0,0,0,0,0,0,0
0008ed71-2,0,0,1,0,0,0,0,0,0,0,1
0008ed71-3,0,0,1,0,1,0,0,0,0,0,1
00091926-1,0,0,0,0,0,0,0,0,0,6,1
00091926-10,1,0,0,0,0,0,0,0,0,2,1


In [85]:
con_conversion = sesiones.loc[sesiones['conversion']>0]
con_conversion_mean = con_conversion.mean()
con_conversion_std = con_conversion.std()
sin_conversion = sesiones.loc[sesiones['conversion']==0]
sin_conversion_mean = sin_conversion.mean()
sin_conversion_std = sin_conversion.std()
comparacion = pd.DataFrame(index=sesiones.columns, columns=['con_conversion_mean'], data=con_conversion_mean)
comparacion['sin_conversion_mean'] = sin_conversion_mean
comparacion['con_conversion_std'] = con_conversion_std
comparacion['sin_conversion_std'] = sin_conversion_std
comparacion

,con_conversion_mean,sin_conversion_mean,con_conversion_std,sin_conversion_std
event,,,,
ad campaign hit,1.269493,0.920486,2.358188,1.527757
brand listing,2.195677,1.017433,5.308486,3.118228
checkout,1.528180,0.281142,1.406608,0.661873
conversion,1.178886,0.000000,0.580345,0.000000
generic listing,1.777390,0.748829,2.561609,1.348529
lead,0.000831,0.004899,0.034107,0.076400
search engine hit,0.742976,0.510642,1.513197,1.133898
searched products,1.228595,0.617287,3.703572,2.391739
staticpage,0.178720,0.050725,0.746444,0.353421
